<a href="https://colab.research.google.com/github/Oliverzera/ProjetoSweetBah/blob/main/Trabalho_Final_Prog_Linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pymprog

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.6 MB/s eta 0:00:00
  Created wheel for pymprog: filename=pymprog-1.1.2-py3-none-any.whl size=43485 sha256=816752dc7103a592123e33ef1d932cb556bbbd4b59436fdd8f5e560b19d42edd
  Stored in directory: /root/.cache/pip/wheels/f7/dc/68/18b668a7137dc2a1e5973c9c3c8543ddcce4daa09a3a8d94b0
Successfully built pymprog


In [ ]:
from pymprog import *

file = open('drive/MyDrive/Dados/inst_25_19.txt','r') # passar o caminho do arquivo smp como 'Dados/' e a instancia q vai usar.
linhas_inst = file.readlines()

# Dados de entrada
H = [2,5,12,15,19,20,26] # Domingos e feriados no mês no estado de SP (em RP)
N = int(linhas_inst[0])  # Número de funcionários
L = int(linhas_inst[1]) # Número mínimo de funcionários em cada turno
NF = 5 # Número de folgas do funcionário durante o mês
M = 30 # Número de dias do mês
FI = [] # Dias escolhidos para folga por cada funcionário

for linha in linhas_inst[2:]:
    valores = linha.split()
    valores_int = [int(valor)-1 for valor in valores]
    FI.append(valores_int)
# --> Da linha 2 pra baixo, esse for aqui vai selecionar as informações de férias e adicionar em uma variavel 'valores_int'

begin("folgas") # Inicia um modelo

# Variáveis de decisão
Z = var("z") # Z sao os dias que o funcionário solicita de folga
A = iprod(range(N), range(M)) # Índices da variável x
X = var("x", A, bool) # Variável x

# Função objetivo: queremos minimizar a quantidade de dias trabalhados dentro dos dias de folga do funcionario
minimize(Z)

#Restrição 1: Cada funcionário deve ter ao menos uma folga em domingos e feriados
for i in range(N):
  sum(X[i,j] for j in H) <= len(H)-1

#Restrição 2: O turno precisa da quantidade mínima de funcionários trabalhando (>= L)
for j in range(M):
  sum(X[i,j] for i in range(N)) >= L

#Restrição 3: Cada funcionário deve ter NF folgas (NF = 5)
for i in range(N):
  sum(X[i,j] for j in range(M)) == M-NF

#Restrição 4: Cada funcionário não pode trabalhar mais do que 5 dias consecutivos
for i in range(N):
  for k in range(25):
    sum(X[i,j] for j in range(k,k+6)) <= 5

#Restrição 5: Busca atender o máximo possível de folgas dos funcionários nos dias solicitados.
#sem essa restrição, ele dá folgas nos dias que achar melhor, não nos dias que foram solicitados
for i in range(N):
  sum(X[i,j] for j in FI[i]) <= Z

# Solução
solve()

print('')
print('Sobre o Resultado abaixo:\n \tQuando o funcionário trabalhou, então 1;' +
      '\n \tCaso ele tenha folgado um dia comum, então 0;' +
      '\n \tSe o funcionário folgar um dia que ele pediu folga, então veremos um +;' +
      '\n \tCaso o funcionário tenha pedido a folga e conseguido, veremos um |; \n')
print(f'Domingos e feriados do mês Novembro/23: {H}')
print(f'Número máximo de folgas não atendidas: {vobj():.0f}')

for i in range(N):
  print(f'Funcionário {i+1:2.0f} : ', end='')
  for j in range(M):
    if j not in FI[i]:
      print(f'{int(X[i,j].primal)} ', end='')
    else:
      if round(X[i,j].primal) == 1:
        print('| ', end='')
      else:
        print('+ ', end='')
  print('')
print('\nNúmero de folgas atendidas')
count = 1
for i in range(N):
  print(f'Funcionário {count:2.0f} = {(4 - sum(X[i,j].primal for j in FI[i])):.0f}')
  count+=1

#| Dia que o funcionário pediu a folga e teve que trabalhar
# + Dia que o funcionario pediu a folga e conseguiu
# 1 Trabalhou, 0 folgou


Sobre o Resultado abaixo:
 	Quando o funcionário trabalhou, então 1;
 	Caso ele tenha folgado um dia comum, então 0;
 	Se o funcionário folgar um dia que ele pediu folga, então veremos um +;
 	Caso o funcionário tenha pedido a folga e conseguido, veremos um |; 

Domingos e feriados do mês Novembro/23: [2, 5, 12, 15, 19, 20, 26]
Número máximo de folgas não atendidas: 3
Funcionário  1 : | + 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 | 1 1 1 0 1 1 1 | 
Funcionário  2 : 0 1 1 1 | 1 + 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 + 1 1 | 1 1 
Funcionário  3 : 1 0 1 | 1 1 1 0 | 1 1 1 1 0 1 1 | 1 1 + 1 1 1 1 1 0 1 1 1 1 
Funcionário  4 : 1 1 | + 1 | 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 | 1 1 
Funcionário  5 : 1 1 1 0 1 1 1 1 1 0 1 1 | 1 1 0 1 1 1 | 1 0 1 1 1 | 1 + 1 1 
Funcionário  6 : 1 | 1 + 1 | | 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 
Funcionário  7 : 1 1 1 1 1 + 1 1 1 1 0 1 | 1 1 0 1 1 1 1 1 0 | 1 1 1 1 0 | 1 
Funcionário  8 : 1 1 | 0 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 | | 1 1 + 1 1 1 
Func

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
